In [1]:
import os
import bokeh
from bokeh.plotting import show
import matplotlib.pyplot as plt
from bokeh.layouts import gridplot
import warnings

from flowio.create_fcs import create_fcs
from scipy.stats import zscore
import flowkit as fk
import pandas as pd
import scanpy as sc

bokeh.io.output_notebook()
%matplotlib inline

_ = plt.ioff()

Loading BokehJS ...

In [2]:
# create result folder
res_dir = '../results/facs_data/'
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
if not os.path.exists(res_dir + 'plates'):
    os.makedirs(res_dir + 'plates')
if not os.path.exists(res_dir + 'xform'):
    os.makedirs(res_dir + 'xform')

In [3]:
# function to get the sort locations on a 384-well plate from the .fcs file
def get_sort_locations(sample):
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
    cols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
            '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
    loc_storages = sorted([x for x in sample.get_metadata().keys() if 'index sorting locations' in x], key=lambda x: int(x.split('_')[-1]))
    inx_locs = []
    for l in loc_storages:
        inx_locs.extend(sample.get_metadata()[l].split(';')[:-1])
    inx_locs = [tuple(map(int, x.split(','))) for x in inx_locs]
    wells = [rows[x[0]] + cols[x[1]] for x in inx_locs]
    
    return wells

In [4]:
# the script works for multiple plates, here we use only one
# sometimes multiple .fcs files are generated for one plate; that was not the case here
# for multiple .fcs files for one plate, the files need to be ordered by the order they were sorted/created
plates = {'index_sort_500ms': '500ms'}

In [5]:
# merge fcs for each plate and extract well info
well_dict = {}
for p in set(plates.values()): # for all plates
    fcs = [x[0] for x in plates.items()]
    samples = [fk.Sample(f'../data/{f}.fcs') for f in fcs] # load the .fcs file as flowkit sample
    wells = [get_sort_locations(s) for s in samples] # lists of lists of the sort locations in the right order
    wells = [l for subl in wells for l in subl] # flatten list
    # are there duplicated wells? I.e. wells with multiple cells sorted
    dupl = set([x for x in wells if wells.count(x) > 1])
    if dupl:
        warnings.warn(f'multiple sorts in {p} well {dupl}')
    well_dict[p] = wells # store sort locations for each plate
    # compensate
    comp_mats = [fk.Matrix(f'spill_{s.original_filename}', spill_data_or_file=s.metadata['spill'], detectors=s.metadata['spill'].split(',')[1:3]) for s in samples] # create compensation matrices from the 'spill' field
    [s.apply_compensation(comp_mats[i]) for i, s in enumerate(samples)] # apply the compensation
    # merge
    df = pd.concat([s.as_dataframe(source='comp') for s in samples])
    # save
    fh = open(res_dir + f'plates/{p}.fcs', 'wb')
    create_fcs(fh, df.values.flatten().tolist(), channel_names=df.columns.get_level_values(0),
               opt_channel_names=['', '', '', '', '', '', '', 'CD38', 'CD34']) # add the surface markers as optional names
    fh.close()

In [6]:
df.columns.get_level_values(0)

Index(['Time', 'FSC-A', 'FSC-W', 'FSC-H', 'SSC-A', 'SSC-W', 'SSC-H', 'PE-A',
       'APC-Cy7-A'],
      dtype='object', name='pnn')

In [7]:
# transform each fcs using the FlowJo biex transform with the standard parameters
session = fk.Session(fcs_samples=res_dir + '/plates/', subsample=False)
transform_dict = {'SSC-A': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'FSC-A': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'FSC-W': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'FSC-H': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'SSC-A': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'SSC-W': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'SSC-H': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0), 
                  'PE-A': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                  'APC-Cy7-A': fk.transforms.WSPBiexTransform('biex', max_value=262144, positive=4.42, width=-100, negative=0),
                 }

for s in session.get_group_samples('default'):
    s.apply_transform(transform_dict)
    df = s.as_dataframe(source='xform').apply(zscore) # apply z-score on the transformed data
    # save
    fh = open(res_dir + f'xform/{s.original_filename}', 'wb')
    create_fcs(fh, df.values.flatten().tolist(), channel_names=df.columns.get_level_values(0),
               opt_channel_names=df.columns.get_level_values(1))
    fh.close()

In [8]:
# load session of the biex transformed and z-scored 
session = fk.Session(fcs_samples=res_dir + 'xform/')
session.get_sample_groups()

['default']

In [9]:
session.get_group_samples('default')

[Sample(v3.1, 500ms.fcs, 9 channels, 336 events)]

In [10]:
# histograms of parameters
for c in s.channels['pnn']:
    ps = []
    for s in session.get_group_samples('default'):
        ps.append(s.plot_histogram(channel_label_or_number=c, source='raw'))
    grid = gridplot(ps, ncols=8, width=200, height=200)
    show(grid)

In [11]:
s.channels['pnn']

0         Time
1        FSC-A
2        FSC-W
3        FSC-H
4        SSC-A
5        SSC-W
6        SSC-H
7         PE-A
8    APC-Cy7-A
Name: pnn, dtype: object

In [12]:
# gating
dim_a = fk.Dimension('APC-Cy7-A', range_min=-0.5, range_max=4) # CD34+
dim_b = fk.Dimension('APC-Cy7-A', range_min=-4, range_max=-1) # CD34-
dim_c = fk.Dimension('PE-A', range_min=None, range_max=None) # all CD38
dim_d = fk.Dimension('PE-A', range_min=None, range_max=-0.5) # CD38-
dim_e = fk.Dimension('PE-A', range_min=-0.3, range_max=None) # CD38+

blasts = fk.gates.RectangleGate('BLAST', parent_gate_name=None, dimensions=[dim_b, dim_c])
lsc = fk.gates.RectangleGate('LSC', parent_gate_name=None, dimensions=[dim_a, dim_d])
prog = fk.gates.RectangleGate('PROG', parent_gate_name=None, dimensions=[dim_a, dim_e])

In [13]:
session.add_gate(blasts)
session.add_gate(lsc)
session.add_gate(prog)
session.analyze_samples()

In [14]:
for s in [s.original_filename for s in session.get_group_samples('default')]:
    ps = []
    for g in ['BLAST', 'LSC', 'PROG']:
        ps.append(session.plot_gate('default', s, g, color_density=False))
        ps[-1].renderers[0].glyph.radius = 0.05
    grid = gridplot(ps, ncols=4, width=400, height=400)
    show(grid)

In [15]:
# compile the facs data
facs_data = {}
for s in session.get_group_samples('default'):
    df = s.as_dataframe(source='raw')
    df.columns = [x.strip() for x in df.columns.get_level_values(1) + ' ' + df.columns.get_level_values(0)] # surface marker + conjugate
    
    df['Well'] = well_dict[s.original_filename.split('.')[0]]
    df['Row'] = df['Well'].apply(lambda x: x[0])
    df['Column'] = df['Well'].apply(lambda x: x[1:])
    
    # annotate final populations
    for pop in ['BLAST', 'LSC', 'PROG']:
        mask = session.get_gate_membership('default', s.original_filename, pop)
        df.loc[mask, 'Population'] = pop
    df['Population'] = df['Population'].replace(pd.NA, 'ungated')
    
    # annotate gate path (only relevant for sequential gating)
    df['Gate Path'] = [[] for i in range(len(df))]
    for g in ['BLAST', 'LSC', 'PROG']:
        mask = session.get_gate_membership('default', s.original_filename, g)
        df.loc[mask, 'Gate Path'].apply(lambda x: x.append(g))
    df['Gate Path'] = df['Gate Path'].apply(lambda x: ";".join(x))
    facs_data[s.original_filename] = df
    
    # transform to normal float
    df = df.astype({k:'float32' for k in df.columns[df.dtypes == 'float64']})
    # save
    df.to_csv(f'../results/facs_data/facs_data_{s.original_filename.split(".")[0]}.txt', sep='\t', index=False)

# from here, copy the facs data .txt files to the data folder

In [16]:
facs_data

{'500ms.fcs':          Time     FSC-A     FSC-W     FSC-H     SSC-A     SSC-W     SSC-H  \
 0   -1.717951 -0.143168 -1.087328  0.944056 -1.469197 -0.305880 -1.624680   
 1   -1.708250  0.223649  1.963795 -1.755856  0.146541  1.655638 -0.576108   
 2   -1.698105 -0.004392 -0.882233  0.912436 -2.147043 -1.603061 -1.846762   
 3   -1.688514  0.788748  0.877482  0.119880  0.740596  0.705785  0.568729   
 4   -1.676714 -0.530861 -0.479894 -0.195032 -1.254147 -1.157720 -0.978128   
 ..        ...       ...       ...       ...       ...       ...       ...   
 331  1.670409 -0.306588 -0.238753 -0.152489  0.564421 -0.570741  0.935409   
 332  1.680090  0.292485  1.019905 -0.679378  0.839736  1.225710  0.451981   
 333  1.689680  1.128395  0.778115  0.668474  1.549425  0.455460  1.651266   
 334  1.699321 -0.559861 -1.191465  0.507379  0.308676  0.277796  0.244681   
 335  1.711000  0.153793 -0.069035  0.273689 -0.471597 -0.103014 -0.518694   
 
      CD38 PE-A  CD34 APC-Cy7-A Well Row Column P

In [18]:
os.system('jupyter nbconvert --to html flowkit_500ms.ipynb --output-dir={}'.format(res_dir))

[NbConvertApp] Converting notebook flowkit_500ms.ipynb to html
[NbConvertApp] Writing 752450 bytes to ../results/facs_data/flowkit_500ms.html


0